<a href="https://colab.research.google.com/github/Yubinex/tensorflow-coursework/blob/master/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow.

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try for yourself!

## Introfuction to Tensors

In [ ]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

In [ ]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

In [ ]:
# Check the dimensions of our vector
vector.ndim

In [ ]:
# Create a matrix (has more than one dimension)
matrix = tf.constant([[10, 7],
                      [7, 10]])
matrix

In [ ]:
# Check the dimensions of our matrix
matrix.ndim

In [ ]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specifiy the data type with dtype parameter
another_matrix

In [ ]:
# Check the dimensions of another_matrix
another_matrix.ndim

In [ ]:
tensor = tf.constant([[[1, 2, 3,],
                       [4, 5, 6,]],
                      [[7, 8, 9,],
                       [10, 11, 12,]],
                      [[13, 14, 15,],
                       [16, 17, 18,]]])
tensor

In [ ]:
tensor.ndim

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an **n-dimensional** array of numbers (where n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.Variable`

In [ ]:
# Create the same tensor with tf.Variable as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

In [ ]:
# Let's try to change one of the elements in our changeable tensor
changeable_tensor[0].assign(7)
changeable_tensor

In [ ]:
# Now let's try to change our unchangeable tensor
#unchangeable_tensor[0].assign(7)
#unchangeable_tensor

# DOES OBVIOUSLY NOT WORK

> 🔑 **NOTE**: Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors

Random tensors are tensors of some abitrary size which contain random numbers.

In [ ]:
# Create two random (but the same) tensors
random_0 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_0 = random_0.normal(shape=(3, 2))
random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3, 2))

# Are they equal?
random_0, random_1, random_0 == random_1

### Shuffle the order of elements in a tensor

In [ ]:
# Shuffle tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7,],
                            [3, 4,],
                            [2, 5,]])
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)

In [ ]:
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level seed

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed:
> If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.

### Other ways to make tensors

In [ ]:
# Create a tensor of all ones
tf.ones([10, 7])

In [ ]:
# Create tensor of all zeros
tf.zeros(shape=(3, 4))

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU computing.

In [ ]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a NumPy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

In [ ]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A)
A, B

### Getting information from tensors

When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [ ]:
# Create a rank 4 tensor
rank_4_tensor = tf.zeros(shape=(2, 3, 4, 5))
rank_4_tensor

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

In [ ]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of the tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor))
print("Total number of elements in our tensor (using `.numpy()`):", tf.size(rank_4_tensor).numpy())

### Indexing tensors

Tensors can be indexed just like Python lists.

In [ ]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

In [ ]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

In [ ]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7],
                             [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

In [ ]:
# Get the last item of each row of our rank 2 tensor
rank_2_tensor[:, -1]

In [ ]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # [...,] is the same as doing [:, :,] here
rank_2_tensor, rank_3_tensor

In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand the final axis

In [ ]:
tf.expand_dims(rank_2_tensor, axis=0) # expand the 0-axis

### Manipulating tensors (tensor operations)

**Basic operations**

`+`, `-`, `*`, `/`

In [ ]:
# Addition
tensor = tf.constant([[10, 7],
                      [3, 4]])
tensor + 10

In [ ]:
# The original tensor is unchanged
tensor

In [ ]:
# Subtraction
tensor - 10

In [ ]:
# Multiplication
tensor * 10

In [ ]:
# Division
tensor / 10

In [ ]:
# We can use the tensorflow built-in function to
tf.multiply(tensor, 10)

> ‼️**Note**: Using the TensorFlow built-in function instead of standard Python operators tends to speed up the code when it's running on a GPU.

**Matrix multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.


>❗There are **two rules** our tensors (or matrices) need to fulfill if we're going to matrix multiply them:
1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

In [ ]:
# Matrix multiplication in tensorflow
print(tensor)
tf.matmul(tensor, tensor)

In [ ]:
# Matrix multiplication with Python operator "@"
tensor @ tensor

In [ ]:
# Create a (3, 2) tensor
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
# Create another (3, 2) tensor
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])
X, Y

In [ ]:
# Try to matrix multiply X by Y (will cause an error)
#X @ Y

In [ ]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2, 3))

In [ ]:
X.shape, tf.reshape(Y, shape=(2, 3)).shape

In [ ]:
# Try to matrix multiply X by reshaped Y again
X @ tf.reshape(Y, shape=(2, 3))

In [ ]:
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

In [ ]:
# Try to change shape of X instead of Y
tf.matmul(tf.reshape(X, shape=(2, 3)), Y)

In [ ]:
# We can do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

In [ ]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`

In [ ]:
X, Y

In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

In [ ]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

In [ ]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

In [ ]:
# Check the values of Y, reshaped Y and transposed Y
print("Normal Y:")
print(Y, "\n")

print("Y reshaped to (2, 3):")
print(tf.reshape(Y, shape=(2, 3)), "\n")

print("Y transposed:")
print(tf.transpose(Y))

In [ ]:
tf.matmul(X, tf.transpose(Y))

Generally when performing matrix multiplication on two tensors end one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors to satisfy the matrix multiplication rules.

### Changing the datatype of a tensor

In [ ]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

In [ ]:
C = tf.constant([7, 10])
C, C.dtype

In [ ]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

In [ ]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

### Aggregating tensor

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [ ]:
# Get the absolute values
D = tf.constant([-7, -10])
D

In [ ]:
# Get the absolute values
tf.abs(D)

Let's go through the following forms of aggregation:
- Get the minimum
- Get the maximum
- Get mean of a tensor
- Get the sum of a tensor

In [ ]:
# Creating a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

In [ ]:
# Find the min
tf.reduce_min(E)

In [ ]:
# Find the max
tf.reduce_max(E)

In [ ]:
# Find the mean
tf.reduce_mean(E)

In [ ]:
# Find the sum
tf.reduce_sum(E)

🛠 **Exercise:** With what we've just learned, find the variance and standard deviation of our `E` tensor using TensorFlow methods.

In [ ]:
# Find the variance
tf.math.reduce_variance(tf.cast(E, tf.float32))

In [ ]:
# Find standard deviation
tf.math.reduce_std(tf.cast(E, tf.float32))

Or using `tensorflow_probability` to do the same:

In [ ]:
import tensorflow_probability as tfp

In [ ]:
tfp.stats.variance(E), tfp.stats.variance(tf.cast(E, tf.float32))

In [ ]:
tfp.stats.stddev(tf.cast(E, tf.float32))

### Find the positional maximum and minimum

In [ ]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

In [ ]:
# Find the positional maximum, get the value at that index, get the maxmimum value and check for equality
tf.argmax(F), F[tf.argmax(F)], tf.reduce_max(F), F[tf.argmax(F)] == tf.reduce_max(F)

In [ ]:
# Find the positional minimum, get the value at that index, get the minimum value and check for equality
tf.argmin(F), F[tf.argmin(F)], tf.reduce_min(F), F[tf.argmin(F)] == tf.reduce_min(F)

### Squeezing a tensor (removing all single dimensions)

In [ ]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

In [ ]:
G.shape

In [ ]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

### One-hot encoding tensors

In [ ]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

# One hot encode our list of indices
tf.one_hot(some_list, depth=4)

In [ ]:
# Rarely used
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="Frieren", off_value="Serie")

### Squaring, power, log, square root

In [ ]:
# Create a new tensor
H = tf.range(1, 10)
H

In [ ]:
# Square it
tf.square(H)

In [ ]:
# Find the power (of 4 in this example)
tf.pow(H, 4)

In [ ]:
# Find the log
tf.math.log(tf.cast(H, tf.float32))

In [ ]:
# Find the square root
tf.sqrt(tf.cast(H, tf.float32))

### Tensors and NumPy

TensorFlow interacts beautifully with NumPy arrays.

> 🔑 **Note:** One of the main differences between a TensorFlow tensor and a NumPy array is that a Tensor can be run on a GPU or TPU (for faster numerical processing)

In [ ]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

In [ ]:
# Convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

In [ ]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

In [ ]:
# Getting a value itself
J = tf.constant([3.])
J.numpy()[0]

In [ ]:
# Default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

### Finding access to GPUs

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

In [ ]:
!nvidia-smi

> 🔑 **Note:**: If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenever possible.